In [11]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [12]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-pz6tmnu_
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-pz6tmnu_
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done


In [13]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [14]:
%%cu
#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <time.h>
#define N 7
__global__ void gpuAdd(int *d_a, int *d_b, int *d_c)
{
    int tid = blockIdx.x;
    if (tid < N)
    d_c[tid] = d_a[tid] + d_b[tid];
}
int main(void)
{
    int h_a[N], h_b[N], h_c[N];
    int *d_a, *d_b, *d_c;
    cudaMalloc((void**)&d_a, N * sizeof(int));
    cudaMalloc((void**)&d_b, N * sizeof(int));
    cudaMalloc((void**)&d_c, N * sizeof(int));
    cudaEvent_t start,end;
    cudaEventCreate(&start);
    cudaEventCreate(&end);
    cudaEventRecord(start);
    for (int i = 0; i < N; i++) 
    {
        h_a[i] = 2*i*i;
        h_b[i] = i ;
    }
    cudaMemcpy(d_a, h_a, N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, N * sizeof(int), cudaMemcpyHostToDevice);
    gpuAdd <<<N, 1 >>>(d_a, d_b, d_c);
    cudaMemcpy(h_c, d_c, N * sizeof(int), cudaMemcpyDeviceToHost);
    printf("Vector addition on GPU \n");
    for (int i = 0; i < N; i++) 
    {
        printf("The sum of %d element is %d + %d = %d\n", i, h_a[i], h_b[i],h_c[i]);
    }
    cudaEventRecord(end);
    cudaEventSynchronize(end);
    float time=0;
    cudaEventElapsedTime(&time, start, end);
    printf("\nexecution time=%f",time);
 
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    return 0;
}

Vector addition on GPU 
The sum of 0 element is 0 + 0 = 0
The sum of 1 element is 2 + 1 = 3
The sum of 2 element is 8 + 2 = 10
The sum of 3 element is 18 + 3 = 21
The sum of 4 element is 32 + 4 = 36
The sum of 5 element is 50 + 5 = 55
The sum of 6 element is 72 + 6 = 78

execution time=0.066464
